In [1]:
import pandas as pd
import numpy as np

In [2]:
#ques1

In [3]:
hostel = 0.60
day_sc = 0.40
A_hostel = 0.30
A_day = 0.20


A = (A_hostel * hostel) + (A_day * day_sc)

host_A = (A_hostel * hostel) / A

host_A

0.6923076923076923

In [4]:
dis = 0.01
sens = 0.99
FPR = 0.02
no_dis = 1 - dis
pt = (sens * dis) + (FPR * no_dis)
d_pt = (sens * dis) / pt
d_pt


0.3333333333333333

In [5]:
#ques2

In [6]:
import matplotlib.pyplot as plt


In [7]:
class NaiveBayesClassifier:
    def __init__(self):
        self.class_probs = {}
        self.feature_probs = {}
        self.classes = None

    def fit(self, X, y):
        self.classes = y.unique()
        self.class_probs = y.value_counts(normalize=True).to_dict()

        self.feature_probs = {}
        for feature in X.columns:
            self.feature_probs[feature] = {}
            for cls in self.classes:
                subset = X[y == cls]
                feature_prob = subset.groupby(feature).size() / subset.shape[0]
                self.feature_probs[feature][cls] = feature_prob.to_dict()

    def predict(self, X):
        predictions = []
        for _, row in X.iterrows():
            class_probs = {}
            for cls in self.classes:
                prob = self.class_probs[cls]
                for feature in X.columns:
                    value = row[feature]
                    feature_prob = self.feature_probs.get(feature, {}).get(cls, {}).get(value, 0)
                    prob *= feature_prob
                print(prob)
                class_probs[cls] = prob
            predicted_class = max(class_probs, key=class_probs.get)
            predictions.append(predicted_class)
        return predictions

data = pd.read_csv('Lab7.csv')

X = data.drop('buys_com', axis=1)
y = data['buys_com']

nb_classifier = NaiveBayesClassifier()
nb_classifier.fit(X, y)

test_data = pd.DataFrame([{
    'age': '<=30',
    'income': 'medium',
    'student': 'yes',
    'credit_rating': 'fair'
}])

predictions = nb_classifier.predict(test_data)
print(f'Prediction for the test data: {predictions[0]}')


0.006857142857142858
0.02821869488536155
Prediction for the test data: yes


In [8]:
#QUES3

In [9]:
class NaiveBayesTextClassifier:
    def __init__(self):
        self.class_probs = {}
        self.word_probs = {}
        self.vocabulary = set()
        self.classes = None

    def fit(self, X, y):
        self.classes = y.unique()
        
        # Calculate class probabilities
        class_counts = y.value_counts()
        total_count = len(y)
        self.class_probs = {cls: count / total_count for cls, count in class_counts.items()}
        
        # Initialize word probabilities
        self.word_probs = {cls: {} for cls in self.classes}
        self.vocabulary = set()
        
        for cls in self.classes:
            subset = X[y == cls]
            words = ' '.join(subset).split()
            self.vocabulary.update(words)
            word_counts = pd.Series(words).value_counts()
            total_words = len(words)
            for word in self.vocabulary:
                # Applying Laplace smoothing
                self.word_probs[cls][word] = (word_counts.get(word, 0) + 1) / (total_words + len(self.vocabulary))

    def predict(self, X):
        predictions = []
        for text in X:
            words = text.split()
            class_probs = {}
            for cls in self.classes:
                prob = self.class_probs[cls]
                for word in words:
                    prob *= self.word_probs[cls].get(word, 1 / (sum(self.word_probs[cls].values()) + len(self.vocabulary)))
                class_probs[cls] = prob
            predicted_class = max(class_probs, key=class_probs.get)
            predictions.append(predicted_class)
        return predictions

df = pd.read_csv('Lab7_2.csv')

# Train the classifier
nb_classifier = NaiveBayesTextClassifier()
nb_classifier.fit(df['Text'], df['Tag'])

# Predict on the entire dataset (since we are not splitting into train/test)
y_true = df['Tag']
y_pred = nb_classifier.predict(df['Text'])
# print(y_pred)


# # Predict the class for a new sentence
test_sentence = ["A very close game"]
predicted_tag = nb_classifier.predict(test_sentence)
# print(predicted_tag)
print(f'The sentence "A very close game" is classified as: {predicted_tag[0]}')

The sentence "A very close game" is classified as: 'Sports'
